<a href="https://colab.research.google.com/github/Athugodage/RuLawSimplification/blob/main/corpus_analysis/text_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Athugodage/RuLawSimplification.git

fatal: destination path 'RuLawSimplification' already exists and is not an empty directory.


In [ ]:
!unzip '/content/RuLawSimplification/corpus/the_corpus.zip'

Archive:  /content/RuLawSimplification/corpus/the_corpus.zip
replace the_corpus.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
import pandas as pd

df = pd.read_csv('the_corpus.csv')

In [ ]:
df.head()

,Название документа,Ссылка,Текст,Комментарий РГ,Date
0,Постановление Конституционного Суда Российской...,https://rg.ru/documents/2022/11/25/document-ob...,Именем Российской Федерации\n Конституционный...,"Бессрочный запрет на профессию для адвокатов, ...",2022-11-25
1,Постановление Пленума Верховного Суда Российск...,https://rg.ru/documents/2022/11/25/document-pl...,В целях обеспечения единства практики применен...,Пленум Верховного суда России утвердил постано...,2022-11-25
2,Федеральный закон от 21 ноября 2022 г. N 455-Ф...,https://rg.ru/documents/2022/11/24/document-po...,Принят Государственной Думой 15 ноября 2022 го...,Единое пособие для беременных женщин и граждан...,2022-11-24
3,Федеральный закон от 21 ноября 2022 г. N 443-Ф...,https://rg.ru/documents/2022/11/24/document-na...,Принят Государственной Думой 10 ноября 2022 го...,На сладкую газировку с 1 июля 2023 года введут...,2022-11-24
4,Федеральный закон от 21 ноября 2022 г. N 448-Ф...,https://rg.ru/documents/2022/11/24/document-by...,Принят Государственной Думой 10 ноября 2022 го...,Приостановленное в этом году бюджетное правило...,2022-11-24


In [ ]:
!pip install razdel
from razdel import sentenize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re
from tqdm import tqdm

tqdm.pandas()

n = 0

def cut_it(text):
    sents = list(sentenize(text))
    start = len(sents) if len(sents) < 40 else 35

    sents = [s.text for s in sents[-start:]] 
    
    new_text = ''
    for s in sents:
        new_text = new_text + ' ' + s
    return new_text
                


df['Cut_original'] = df['Текст'].progress_apply(cut_it)


100%|██████████| 2022/2022 [00:19<00:00, 103.61it/s]


In [ ]:
def cut_it(text):
    ## функции отличаются, т.к. в первая режет со второго предложения,
    ## а вторая с первого. Также тут меньше предложений в итоге

    sents = list(sentenize(text))
    end = len(sents) if len(sents) < 30 else 21
    if len(sents) > 30:
        end = 21
    elif len(sents) < 30:
        end = len(sents)

    sents = [s.text for s in sents[0:end]] 
    
    new_text = ''
    for s in sents:
        new_text = new_text + ' ' + s
    return new_text
                
df['Cut_comment'] = df['Комментарий РГ'].progress_apply(cut_it)

100%|██████████| 2022/2022 [00:01<00:00, 1568.89it/s]


In [ ]:
data = df.copy()

df[['Cut_original', 'Cut_comment']]

,Cut_original,Cut_comment
0,"При этом должны использоваться не чрезмерные,...","Бессрочный запрет на профессию для адвокатов,..."
1,"Суд, установив факт нарушения прав потребител...",Пленум Верховного суда России утвердил постан...
2,"В состав семьи, учитываемый при определении п...",Единое пособие для беременных женщин и гражда...
3,"2019, N 50, ст. 7201; 2022, N 27, ст. 4607) с...",На сладкую газировку с 1 июля 2023 года введу...
4,Перечисление средств по таким контрактам (дог...,Приостановленное в этом году бюджетное правил...
...,...,...
2017,"2006, N 52 (часть I), ст. 5498; 2007, N 46, с...",Через десять дней автомобиль можно будет пост...
2018,Правительство Российской Федерации постановля...,Правительство утвердило изменения в правила н...
2019,Правительство Российской Федерации постановля...,"Правительство приняло постановление, расширяю..."
2020,Правительство Российской Федерации постановля...,Специфическим образом теперь должны быть раск...


In [ ]:
df[['Cut_original', 'Cut_comment']].to_csv('cut_corpus.csv', index=False)